In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)


height has been deprecated.



In [2]:
df_train = pd.read_csv('../data/train_data3.csv', nrows=3000)
df_train.head()

   Unnamed: 0  genre_id            ts_listen  media_id  album_id  context_type         release_date  platform_name  platform_family  media_duration  listen_type  user_gender  user_id  artist_id  \
0           0     25471  2016-12-01 11:00:15    222606     41774            12  2004-07-04 00:00:00              1                0             4.0            0            0     9241      55164   
1           1     25571  2016-11-30 20:25:35    250467     43941             0  2006-03-01 00:00:00              2                1             3.0            0            0    16547      55830   
2           2        16  2016-11-19 11:59:13    305197     48078             1  2014-07-14 00:00:00              2                1             2.0            1            1     7665       2704   
3           3         7  2016-11-26 07:21:38    900502     71521             0  2000-10-30 00:00:00              0                0             4.0            0            1     1580        938   
4           4  

### Quantidade de músicas diferentes que ele ouviu no período de treino (em Flow e não Flow)

In [3]:
users_id = df_train["user_id"].unique()
dict_user = {}

In [4]:
# df_train[(df_train["user_id"] == 1)]["media_id"].value_counts().values[0]

In [5]:
def stats_musics_listened(user_id, dict_user):
    df_user = df_train[(df_train["user_id"] == user_id)].copy()
    
    list_feats = ["mean_media_duration_not_listened", 
                  "mean_media_duration_listened",
                  "mean_media_age_not_listened",
                  "mean_media_age_listened"]
    
    mean_duration = df_user[(df_user["is_listened"] == 0)]["media_duration"].mean()
    mean_duration_listened = df_user[(df_user["is_listened"] == 1)]["media_duration"].mean()
    mean_age = df_user[(df_user["is_listened"] == 0)]["diff_ts_listen_AND_release_date_Y"].mean()
    mean_age_listened = df_user[(df_user["is_listened"] == 1)]["diff_ts_listen_AND_release_date_Y"].mean()
    
    
    # df_user[list_feats] = df_user.apply(lambda row: pd.Series([mean_duration, mean_duration_listened, mean_age, mean_age_listened]), axis=1)
    # df_user.set_index("user_id", inplace=True)
    # df_user.fillna(0, inplace=True)
    # return df_user[list_feats].iloc[0:1]

    dict_user[user_id].update({"mean_media_duration_not_listened": mean_duration,
                               "mean_media_duration_listened": mean_duration_listened,
                               "mean_media_age_not_listened": mean_age,
                               "mean_media_age_listened": mean_age_listened})
    return dict_user


In [6]:
def number_musics_listened(user_id, dict_user):
    df_user = df_train[(df_train["user_id"] == user_id)]
    max_repet = df_train[(df_train["user_id"] == 1)]["media_id"].value_counts().values[0]
    # case where is_listened was not fixed
    with_flow = len(df_user[(df_user["listen_type"] == 1)]["media_id"].unique())
    regular = len(df_user[(df_user["listen_type"] == 0)]["media_id"].unique())
    
    # case where is_listened equals to 1
    with_flow_listened = len(df_user[(df_user["listen_type"] == 1) & (df_user["is_listened"] == 1)]["media_id"].unique())
    regular_listened = len(df_user[(df_user["listen_type"] == 0) & (df_user["is_listened"] == 1)]["media_id"].unique())
    dict_user[user_id].update({"nmidia_with_flow": with_flow, "nmidia_regular": regular, "nmidia_with_flow_listened": with_flow_listened,
                          "nmidia_regular_listened": regular_listened, "nmidia_max_repet": max_repet})
    return dict_user

def number_of_artists_listened(user_id, dict_user):
    df_user = df_train[(df_train["user_id"] == user_id)]
    # case where is_listened was not fixed
    with_flow = len(df_user[(df_user["listen_type"] == 1)]["artist_id"].unique())
    regular = len(df_user[(df_user["listen_type"] == 0)]["artist_id"].unique())
    
    # case where is_listened equals to 1
    with_flow_listened = len(df_user[(df_user["listen_type"] == 1) & (df_user["is_listened"] == 1)]["artist_id"].unique())
    regular_listened = len(df_user[(df_user["listen_type"] == 0) & (df_user["is_listened"] == 1)]["artist_id"].unique())
    dict_user[user_id].update({"nartist_with_flow": with_flow, "nartist_regular": regular, "nartist_with_flow_listened": with_flow_listened,
                          "nartist_regular_listened": regular_listened})
    return dict_user

def number_of_platform(user_id, dict_user):
    df_user = df_train[(df_train["user_id"] == user_id)]
    # case where is_listened was not fixed
    with_flow = len(df_user[(df_user["listen_type"] == 1)]["platform_name"].unique())
    regular = len(df_user[(df_user["listen_type"] == 0)]["platform_name"].unique())
    
    # case where is_listened equals to 1
    with_flow_listened = len(df_user[(df_user["listen_type"] == 1) & (df_user["is_listened"] == 1)]["platform_name"].unique())
    regular_listened = len(df_user[(df_user["listen_type"] == 0) & (df_user["is_listened"] == 1)]["platform_name"].unique())
    dict_user[user_id].update({"nplatform_with_flow": with_flow, "nplatform_regular": regular, "nplatformt_with_flow_listened": with_flow_listened,
                          "nplatform_regular_listened": regular_listened})
    return dict_user

In [7]:
for ind, user_id in enumerate(users_id):
    dict_user[user_id] = {}
    dict_user = number_musics_listened(user_id, dict_user)
    dict_user = number_of_artists_listened(user_id, dict_user)
    dict_user = number_of_platform(user_id, dict_user)

    dict_user = stats_musics_listened(user_id, dict_user)
    if ind % 1000 == 0:
        print("Restam: {}".format(len(users_id) - ind))

Restam: 319


Restam: 1319


In [8]:
df_user_feat = pd.DataFrame.from_dict(dict_user, orient="index")
df_user_feat.reset_index(level=0, inplace=True)
#df_user_feat.columns = ["user_id"] + ["nmidia_" + col for col in df_user_feat.columns[1:].tolist()]
df_user_feat.to_csv("test_user.csv", sep=";", index=None)

In [9]:
df_user_feat.head()

   index  nartist_with_flow  nartist_with_flow_listened  nplatform_regular_listened  nmidia_regular_listened  mean_media_duration_not_listened  nmidia_regular  nplatform_regular  nmidia_with_flow  \
0      1                  1                           1                           1                        1                               NaN               1                  1                 1   
1      3                  0                           0                           1                        3                               NaN               3                  1                 0   
2      4                  1                           1                           0                        0                               4.4               0                  0                 2   
3      5                  0                           0                           0                        0                               3.0               1                  1                 0   
4    

In [17]:
df_user_feat["nmidia_with_flow_listened_PROP"] = df_user_feat["nmidia_with_flow_listened"] / df_user_feat["nmidia_with_flow"]
df_user_feat["nmidia_regular_listened_PROP"] = df_user_feat["nmidia_regular_listened"] / df_user_feat["nmidia_regular"]
df_user_feat["nmidia_PROP_diff"] = df_user_feat["nmidia_with_flow_listened_PROP"] / df_user_feat["nmidia_regular_listened_PROP"]

df_user_feat["nartist_with_flow_listened_PROP"] = df_user_feat["nartist_with_flow_listened"] / df_user_feat["nartist_with_flow"]
df_user_feat["nartist_regular_listened_PROP"] = df_user_feat["nartist_regular_listened"] / df_user_feat["nartist_regular"]
df_user_feat["nartist_PROP_diff"] = df_user_feat["nartist_with_flow_listened_PROP"] / df_user_feat["nartist_regular_listened_PROP"]

In [18]:
df_user_feat.to_csv("nmidia_feats.csv", sep=";", index=None)